CompoundCalulator 20200816
==========================
Given a list of base compound names and molecular weights, this notebook calculates possible derivatives (e.g. metabolites) in 2 phases, and adds adducts and losses resulting in a list of possible ion masses with labels.
The intention is that this be used with the PCVG compound finding code, and the idea is that once we have a suggested identification, e.g. ibuprofen, we can calculate other potential masses and look for matches with the ions we've found. This eliminates a lot of manual calculations and allows interactive investigation, e.g. if we find an adduct that corresponds to addition of C2H3O2Na we can quickly incorporate it into our lists and see if it shows up elsewhere. It also allows us to look for forms that we might not otherwise consider.

To do
----
- Add UI - a form?
- Clean up and standardize code
- Refactor the Composition class to use Count and include adduct, etc.


Changes
-------

20201007

All user-defined parameters accessible in "Setup"...code can be executed with "Run selected cell and all below"
20201005

Modifications are separated from the base compound with underscores so that only losses have a minus sign making the putput easier to read.

The base compound information is now a list of (name, mass) tuples which provides a way to incorporate specific known modifications. For example, vinpocetin shows a loss of C2H4, possibly following oxidations, which then undergoes further pahase 1 and 2 modifications so the base_compound list contains both Vinpo and Apo-vinpo.

The output can now include hereodimers which are calculated by generating new compounds from all pairs of modified compounds, i.e. after the phase 1 and phase 2 nodifications have been generated but prior to generating the final list with adducts. NOTE: this can generate a lot of extra entries!

In [1]:
from itertools import combinations
from itertools import groupby

Class and function definitions
------------------------------
The basic entity is a 'Composition'...NB. this is not an elemental composition but simply a text label, a count and a mass. When compositions are combined, the labels are concatenated (using a specified separator character) and the masses are added.

In [178]:
from dataclasses import dataclass

@dataclass
class Composition:
    Name: str = ""
    Count: int = 1
    Mass: float=-1
    Root:  str = ""       #the composition this on is based on 
    
    mods = {}
    
    def __init__(self, name, count, mass=None, root=None):
        self.Name = f'{name}' if count == 1 else f'{count}({name})'
        self.Count = 1    # there's only one of these even if the 'count' (really a multiplier) is gretaer
        self.Mass = mass if mass else self.Mods[name]*count        
        
        if root:
            self.Root = root
        else:
            self.Root = name
    
    # Make the Composition from a (Name, Count) tuple
    @classmethod
    def from_tuple(cls, t):
        return Composition(t[0],t[1])

    # make a composition from a list of (Name,Count)tuples
    @classmethod
    def from_tuple_list(cls, t_list):
        comp = None
        
        for t in t_list:           
            if not comp:               
                comp = Composition.from_tuple(t)    #create a comp from the first in the list so we can append others to it
            else:
                comp2 = Composition.from_tuple(t)
                comp = comp.add_comp(comp2, sep='.')
                
        return comp
    
    @classmethod
    def proton(cls):
        return Composition('H+', 1, 1.00727)
    
    def protonate(self):
        return self.add_comp(Composition('H+', 1, 1.00727), sep='.')
    
    def deprotonate(self):
        return self.add_comp(Composition('[-H+]-', 1, -1.00727), sep='.')

    def make_copy(self, mult=1):
        return Composition(self.Name, self.Count*mult, self.Mass*mult, self.Root)
    
    def label(self):
        return self.Name
    
    # Merge two compositions to generate a new one with a mass
    def add_comp(self, comp1, sep='_'):
        new_name = self.label() + sep + comp1.label()
        new_mass = self.Mass + comp1.Mass
        return Composition( new_name, 1, root=self.Root, mass=new_mass)

    
    # expands a composition by generating a list of Name repeated Count times
    def expand(self):
        result = [self.Name for n in range(self.Count)]
        return result

#provide the set of allowed changes
Composition.Mods = {'OH':15.99492,
        'COOH':29.97418,     #COOH is CH3->COOH, i.e. +O2, -H2)
        'gluc':176.032088,
        'sulphate':79.956815,
        'NH3':17.026549,     #adducts from here
        'Na-H':21.981944,
        'K-H':37.955881,
        'K41-H': 39.9540,
        'Ca-2H': 37.946941,
        'H2O':-18.010565,
        'NaAc': 82.003074,
        'NaFo': 67.987424,     # sodium formate
        'C2H4O2':60.021129,
        'CH2O2':46.004931,
        'CO2':-43.989829,
        'Am':-17.026549}
 
print('Proton: ', Composition.proton())

a = Composition('Na-H',2)
print('Normal init:', a)

b = Composition.from_tuple(('K-H',2))
print('From tuple:', b)

ab = a.add_comp(b, sep='.')
print('From merge:', ab)

print('Expanded:', a.expand())

t_list = [('Na-H',2),('K-H',2), ('NH3', 1)]
abc = Composition.from_tuple_list(t_list)
print('From tuple list:', abc)

Proton:  Composition(Name='H+', Count=1, Mass=1.00727, Root='H+')
Normal init: Composition(Name='2(Na-H)', Count=1, Mass=43.963888, Root='Na-H')
From tuple: Composition(Name='2(K-H)', Count=1, Mass=75.911762, Root='K-H')
From merge: Composition(Name='2(Na-H).2(K-H)', Count=1, Mass=119.87565, Root='Na-H')
Expanded: ['2(Na-H)']
From tuple list: Composition(Name='2(Na-H).2(K-H).NH3', Count=1, Mass=136.902199, Root='Na-H')


In [179]:
def make_combinations(limit_list, max_combinations):
    """Given a list of limits as tuples (comp, upper_limit), return all combinations to a given maximum value
    """

    entities = []

    # Use the limit_list to generate an expanded list of individual entities, i.e. [('X', 2), ('Y',2)] +> X, X, Y, Y
    for l in limit_list:
        for n in range(l[1]):
            entities.append(l[0])

    entity_combinations = []

    # Now find all combinations of 1 entity, 2 entities...to the max number required
    # This will include duplicates, e.g. x,y and y,x
    for i in range(1, max_combinations + 1): 
        entity_combinations += list(combinations(entities, i))

    # making this into a set will find the unique combinations.
    # initially each combination tuple was sorted to make sure they were canonicalized, but this doesn't seem to be needed
    # i.e. combs = [tuple(sorted(c)) for c in entity_combinations]; cel = set(combs)
    
    csl=set(entity_combinations)   

    csa = []

    # we convert these back into the form ('x',2)('y,'1) by grouping the elements of each combination
    # and recording the element and its count...Note each group has to be converted to a list for this to work
    for c in csl:
        csa.append( [(key, len(list(group))) for key, group in groupby(c)])
    
    return csa
        
combs = make_combinations([('y',3), ('x',2), ('z',2)], 3)
print(len(combs),'should be 17')   #...should be 17

17 should be 17


In [180]:
def add_mods(compounds, limits, sep='_'):
    """
    Adds modifications to each compound in the list returning the new compound list.
    The modfications are provided as a list of (mods, max count) tuples
    """
    mods = []

    # Make the compounds by copying the base and adding the possible mods
    for c in compounds:
        for l in limits:
            for i in range(l[1]):
                new_comp = c.make_copy().add_comp(Composition(l[0], i+1), sep=sep)
                mods.append(new_comp)
                #print(new_comp)

    compounds += mods
    
    return compounds


Setup
-----

Provide the  base compound information and other parameters.
The base compounds are supplied as a list of (name, mass) tuples.
As shown below, the mass need not be a real known compound but can be an observed and unexplained peak so that it's potential derivatives are generated.

All user-defined parameters are set here so, once they are set, the code can be executed with 'Run selected cell and all below"

In [181]:
import os

# Define the compound(s) we want to work with
#
base_compounds = [('Guan', 283.091669),  # Guanosine
                  ('F', 151.0489),       # fragment-H+
                  ('x678', 678.5042),    # unknown at 679.5115 - H+
                  ('Form', 46.005479),   # formic acid
                  ('Et3N', 101.120449)]  # tri-ethyl ammonia   

#base_compounds = [('Ibu', 206.1307)]   #Identifier + MW
#base_compounds = [('Vinpo', 350.1994),('Apo_vinpo', 322.1681)]  #Apo_vinpo is vinpo-C2H4
#base_compounds = [('x543', 543.2068+1.00727)] 

multimer_limit = 3               # maximum multimer count
max_adduct_count = 7             # total number of adducts allowed
ionization = 'positive'          # only 'negative' changes the settings...anything else is 'positive'
include_hetero_dimers = True     # if True, calculate dimers of *different* compounds

# Output parameters
# the default file name is based on the first compound name and the polarity selected
base_name = base_compounds[0][0]  #Extract the name of the first compound to use for file naming

output_mass_limit = 1000    # masses greater than this are not written to the file
xic_width = 0.0             # if 0 the normal output form is used...alternative, e.g. 0.01, to generate the PeakView compatible form
write_locally = False       # write to the same location as the notebook (useful for Colab); if 'False' a file path is required

# define file path (used if write_locally = False)
# this is a convenient platform independent way to provide file paths as a list of directories
f_dir = os.sep + os.path.join('Users','ronbonner','Data','PCA')

# Define the limits for metabolites and adducts...
# Defining this way is not required but allows metabolite and adduct sets to be easily changed depending on polarity.

phase1_limits = [('OH', 0), ('COOH', 0)]  # metabolite modifications - phase 1

if ionization == 'negative':
    phase2_limits = [('gluc', 1)] #, ('sulphate', 1)]
    adduct_limits = [('Na-H', 3), ('K-H', 2), ('NaAc',2), ('NaFo', 1)]  
    loss_limits = [('H2O',2), ('CO2',1)]
else:
    phase2_limits = [('gluc', 0)]
    adduct_limits = [('Na-H', 3), ('K-H', 3), ('K41-H', 1), ('NH3', 0), ('CH2O2',2), ('C2H4O2',1)]
    loss_limits = [('H2O',1), ('Am', 1)]

Step 1 - Adduct generation
---------------------------

Generate a list of possible adduct forms by generating all comibnations of adducts (up to the specified limit) and selecting the unique forms (i.e. as far as we are concerned, a+b+a is the same as a+a+b). Note: this approach would also work if we wanted to allow combinations of the metabolites. These will be added to each compound.

In [182]:
adduct_combs = make_combinations(adduct_limits, 4)
    
adduct_comps = [Composition.from_tuple_list(c) for c in adduct_combs]
adduct_comps = sorted(adduct_comps, key=lambda x: x.Mass)

print(len(adduct_comps),'adduct forms')

# for ac in adduct_comps:
#     print(ac)

76 adduct forms


Step 2 - Compound generation
-----------------------------

We convert the base compound list to a of compositions and then successively apply the various modifications, generating extended compound lists, as follows

- phase 1
- phase 2

Then calculate the dimers and heterodimers (if desired)

In [183]:
# Make the compounds by copying the base and adding the possible mods

compounds = [Composition(name, 1, mass) for name, mass in base_compounds]
        
compounds = add_mods(compounds, phase1_limits)
print(len(compounds), 'after phase 1')

compounds = add_mods(compounds, phase2_limits)
print(len(compounds), 'after phase 2')

multimers = []

for c in compounds:
    for m in range(2, multimer_limit+1):
        new_comp = c.make_copy(m)
        multimers.append(new_comp)

if include_hetero_dimers:
    for i, c in enumerate(compounds):
        for j in range(i+1, len(compounds)):
            new_comp = c.make_copy()
            new_comp_2 = compounds[j].make_copy()
            new_comp = new_comp.add_comp(new_comp_2, sep='+')
            multimers.append(new_comp)
    
compounds += multimers

print(len(compounds), 'with multimers')

compounds = add_mods(compounds, loss_limits, sep='-')
print (len(compounds), 'after losses')

for c in compounds:
    print(c)

5 after phase 1
5 after phase 2
25 with multimers
75 after losses
Composition(Name='Guan', Count=1, Mass=283.091669, Root='Guan')
Composition(Name='F', Count=1, Mass=151.0489, Root='F')
Composition(Name='x678', Count=1, Mass=678.5042, Root='x678')
Composition(Name='Form', Count=1, Mass=46.005479, Root='Form')
Composition(Name='Et3N', Count=1, Mass=101.120449, Root='Et3N')
Composition(Name='2(Guan)', Count=1, Mass=566.183338, Root='Guan')
Composition(Name='3(Guan)', Count=1, Mass=849.2750070000001, Root='Guan')
Composition(Name='2(F)', Count=1, Mass=302.0978, Root='F')
Composition(Name='3(F)', Count=1, Mass=453.1467, Root='F')
Composition(Name='2(x678)', Count=1, Mass=1357.0084, Root='x678')
Composition(Name='3(x678)', Count=1, Mass=2035.5126, Root='x678')
Composition(Name='2(Form)', Count=1, Mass=92.010958, Root='Form')
Composition(Name='3(Form)', Count=1, Mass=138.016437, Root='Form')
Composition(Name='2(Et3N)', Count=1, Mass=202.240898, Root='Et3N')
Composition(Name='3(Et3N)', Count=

Step 3 - Generate ion forms
----------------------------

We now add all the adduct forms to each of the compounds. The approach relies on adducts being formed by replacing labile protons and are therefore indpendent of the polarity; the final form is determined by providing a charge agent, i.e. adding or subtracting protons.


In [184]:
ion_forms = []  

for c in compounds:
    
    # add the base compound, with a proton added or subtracted deopending on the ionization mode
    new_comp = c.make_copy()
    if ionization == 'negative':
        new_comp = new_comp.deprotonate()
    else:
        new_comp = new_comp.protonate() 
        
    ion_forms.append(new_comp)   
    
    # then add the adduct forms
    for a in adduct_comps:
        new_comp = c.make_copy().add_comp(a, sep='.')
        if ionization == 'negative':
            new_comp = new_comp.deprotonate()
        else:
            new_comp = new_comp.protonate()            
        ion_forms.append(new_comp)       
        
print(len(ion_forms))
# for ion in sorted(ion_forms, key=lambda x: x.Mass):
#     print(ion)

5775


Step 4 - Save the mass/name list
--------------------------------

Optionally save the ion forms as a simple tab delimited text file.
- the main format is: mass, label
- an additional format: mass, xic width, name is intended to be used with PeakView Extract XIC (by importing it)

The list can also be truncated to an upper mass limit.

To be sure the file exists, we re-open it and count the nuber of lines

In [185]:
save_file = False

if save_file:

    line_count = 0

    ion_forms = sorted(ion_forms, key=lambda x: x.Mass)

    if ionization == "negative":
        f_name = f'{base_name} ions neg.txt'
    else:
        f_name = f'{base_name} ions pos.txt'

    output_path = f_name if write_locally else os.path.join(f_dir, f_name)

    print (output_path)

    with open(output_path, 'w') as f:

        for ion in ion_forms:

            if ion.Mass > output_mass_limit: 
                break       

            if xic_width:
                f.write('{:10.4f}\t{}\t{}\n'.format(ion.Mass, xic_width,ion.Name))
            else:
                f.write(f'{ion.Mass:10.4f}\t{ion.Name}\n')

            line_count += 1

        f.close()

    print(line_count, 'lines written to', output_path)
    
    with open(output_path, 'r') as f:    
        lines_read = f.readlines()    
        f.close()

    print(len(lines_read), 'read')

# Step 5 - Matching

Compare the list we generated to an input peak list (assumed to be mass and intesity)

In [186]:
import os

 # Set up fie names and paths...this is platform independent (i.e. we don't need to know the separator character)
f_dir = os.sep + os.path.join('Users','ronbonner','Desktop')

peak_file = "201023 Erngren Peaklist_guanosine - no annotation.txt"

peak_file_path = os.path.join(f_dir, peak_file)

peaks = []    #list of (mass, inten) tuples

with open(peak_file_path, 'r') as f:  
    lines = f.readlines()   
    f.close()

for line in lines:
    parts = line.split()
    peaks.append((float(parts[0]), float(parts[1])))
    
peaks = sorted(peaks, key = lambda x: x[0])     # ensure the list is sorted by mass

masses, intens = zip(*peaks)

tic, base_peak_inten = sum(intens), max(intens)

print(f'{len(peaks)}, peaks read. TIC {tic}, base peak inten {base_peak_inten}')


1862, peaks read. TIC 266939600.0, base peak inten 30100000.0


In [187]:
ions = sorted(ion_forms, key = lambda x: x.Mass)   # sort values by mass to check for matches...

peak_index, ion_index, peaks_matched = 0, 0, 0
peak_half_window = 0.01
matches = []

# Loop all the values and peaks looking for matches within the specified window
while (ion_index < len(ions)) and (peak_index < len(peaks)):

    this_peak, this_ion = peaks[peak_index], ions[ion_index]
    low_peak, high_peak = this_peak[0]-peak_half_window, this_peak[0]+peak_half_window
  
    # Increment the vaue if it's too low and the peak if the value is too high
    if this_ion.Mass < low_peak:
        ion_index += 1
        continue

    if this_ion.Mass > high_peak:
        peak_index += 1
        continue

    # save matching pair and peak
    # since there may be more than one peak that matches this ion value, we look ahead at the peaks
    # we use a separate index for this so the current peak can be used with the next vion alue
    # we also track the ions matched since some ions may have more than one match
    matches.append((peak_index, this_peak[0], this_ion.Mass-this_peak[0], this_peak[1], this_ion.Mass, this_ion.Name))
    look_ahead = peak_index + 1
    look_ahead_peak = peaks[look_ahead]
    peaks_matched += 1
    
    # look ahead at the peaks while they're still within the search window and add any matches to the list
    # increment look ahead to the next peak
    while (look_ahead_peak[0] - this_ion.Mass) < peak_half_window:
        matches.append((look_ahead, this_ion.Mass, this_ion.Mass-look_ahead_peak[0], look_ahead_peak[1], this_ion.Mass, this_ion.Name))
        look_ahead +=1
        look_ahead_peak = peaks[look_ahead]

    ion_index += 1 # increment ion index but not peak_index - there may be more than one within the window..

matched_indices = list(set([m[0] for m in matches]))   # get the unique indices since a peak may have more than one match

matched_indices = sorted(matched_indices)

matched_inten = sum([peaks[i][1] for i in matched_indices])  # sum the intensities

percent_matched = matched_inten * 100/tic

print(f'{len(matched_indices)} peaks matched ({percent_matched:.1f}% tic), {len(matches)} total matches from {len(peaks)} peaks')

#a match = (peak index, peak mass, mass delta, peak inten, ion mass, ion name)

289 peaks matched (61.9% tic), 371 total matches from 1862 peaks


In [188]:
# Look for C13 isotopes of matched peaks
#matches = sorted(matches, key = lambda x: x[1])  # sort by mass to look for 13C...

c13_matches = []

last_matched_mass = 0
c13_half_window = 0.005
max_C13_count = 4

for peak_index in list(matched_indices):    #only need to look at each peak once
    
    m_mass = peaks[peak_index][0]
    
#     print('m:', peak_index, m_mass)
    
    next_peak_index = peak_index   #start looking at the next higher peak
        
    keep_going = True
    
    for c13_count in range(1, max_C13_count+1):  #look for 1,2,3... C13
    
        c13_mass = m_mass + (c13_count * 1.004)   #expected c13 mass
        
#         print("13c:", c13_count, c13_mass)

        #while v_mass < (c13_mass + c13_half_window):
        while next_peak_index < len(peaks) - 1:
            
            next_peak_index += 1  # point at next value in list
                
            next_peak_mass, next_peak_inten = peaks[next_peak_index]
            
#             print("   ", next_peak_index, next_peak_mass)
            if next_peak_mass > (c13_mass + c13_half_window):
                keep_going = False       # when one isotope is not matched we abort and stop looking for more
                break
                
            if next_peak_mass > (c13_mass - c13_half_window):
#                 print( "  13 match", c13_count, m_mass, next_peak_mass)
                c13_name = f'{m_mass:.4f}(+{c13_count})'
                c13_matches.append((next_peak_index, next_peak_mass, next_peak_mass-c13_mass, next_peak_inten, c13_mass, c13_name))
                break
        
        if not keep_going:
#             print(m_mass, c13_count, c13_mass, 'skipped')
            break     # leave 13c for loop

print(len(c13_matches),'C13 matches')
           
# for c13m in c13_matches:
#     print(c13m)


122 C13 matches


In [189]:
matches += c13_matches

matches = sorted(matches, key = lambda x: x[1])  

matched_indices = set([m[0] for m in matches])   # get the unique indices since a peak may have more than one match

matched_inten = sum([peaks[i][1] for i in matched_indices])  # sum the intensities

percent_matched = matched_inten * 100/tic

print(f'{len(matched_indices)} peaks matched ({percent_matched:.1f}% tic), {len(matches)} total matches from {len(peaks)} peaks')

367 peaks matched (64.8% tic), 493 total matches from 1862 peaks


In [194]:
def match_as_str(m):
    return f'{m[0]:5}: {m[1]:12.4f} ({m[2]*1000:6.1f}) {m[3]:12.1f} {m[4]:10.4f} {m[5]}'

for m in matches:
    print(match_as_str(m))

    8:      90.9762 (   0.4)     239000.0    90.9766 Form.2(Na-H).H+
   17:     102.1272 (   0.5)    1480000.0   102.1277 Et3N.H+
   20:     103.1304 (  -0.8)     123000.0   103.1312 102.1272(+1)
   25:     106.9501 (   0.5)     387000.0   106.9506 Form.Na-H.K-H.H+
   26:     107.0850 (  -1.9)      20900.0   107.0831 Et3N-Am.Na-H.H+
   28:     108.9479 (   0.8)      33100.0   108.9487 Form.Na-H.K41-H.H+
   51:     122.9241 (   0.4)    1880000.0   122.9245 Form.2(K-H).H+
   55:     123.9271 (  -1.0)      21500.0   123.9281 122.9241(+1)
   58:     124.9222 (   0.4)     277000.0   124.9226 Form.K-H.K41-H.H+
   68:     134.0455 (   0.1)      58700.0   134.0456 F-H2O.H+
   72:     135.0296 (  -1.4)    1900000.0   135.0282 Form-H2O.CH2O2.C2H4O2.H+
   72:     135.0296 (  -0.8)    1900000.0   135.0288 2(Form)-H2O.C2H4O2.H+
   72:     135.0296 (   0.0)    1900000.0   135.0296 F-Am.H+
   75:     136.0132 (  -0.9)      97600.0   136.0123 Form-Am.CH2O2.C2H4O2.H+
   75:     136.0132 (  -0.4)      9

In [191]:
peak_matching = [True if i in matched_indices else False for i in range(len(peaks))]

unmatched = [peaks[i] for i in range(len(peaks)) if not peak_matching[i]]

one_percent = base_peak_inten / 100

for p in unmatched:
    m,inten = p
    percent_base_peak = inten * 100/ base_peak_inten
    if inten > one_percent:
        print(f'{m:10.4f} {inten:10.0f} {percent_base_peak:8.2f}% base peak')

  103.9550     385000     1.28% base peak
  118.0857     561000     1.86% base peak
  118.1218     523000     1.74% base peak
  141.1127     320000     1.06% base peak
  155.1286     883000     2.93% base peak
  156.0415     601000     2.00% base peak
  217.1040     351000     1.17% base peak
  230.2476     943000     3.13% base peak
  267.1693     630000     2.09% base peak
  273.1669     724000     2.41% base peak
  277.1044     493000     1.64% base peak
  279.0946     369000     1.23% base peak
  288.2896     379000     1.26% base peak
  313.1427     330000     1.10% base peak
  326.8861     321000     1.07% base peak
  342.8600     328000     1.09% base peak
  365.1551     583000     1.94% base peak
  378.8997     418000     1.39% base peak
  394.8739     788000     2.62% base peak
  410.8474     714000     2.37% base peak
  426.8215     344000     1.14% base peak
  429.2405     438000     1.46% base peak
  441.1886     378000     1.26% base peak
  443.2182     485000     1.61% ba

In [192]:
for ion_form in ions:
    print(f'{ion_form} if ion_form.Mass < 200')

Composition(Name='Form-H2O.H+', Count=1, Mass=29.002184, Root='Form') if ion_form.Mass < 200
Composition(Name='Form-Am.H+', Count=1, Mass=29.986200000000004, Root='Form') if ion_form.Mass < 200
Composition(Name='Form.H+', Count=1, Mass=47.012749, Root='Form') if ion_form.Mass < 200
Composition(Name='Form-H2O.Na-H.H+', Count=1, Mass=50.984128, Root='Form') if ion_form.Mass < 200
Composition(Name='Form-Am.Na-H.H+', Count=1, Mass=51.968144, Root='Form') if ion_form.Mass < 200
Composition(Name='Form-H2O.K-H.H+', Count=1, Mass=66.958065, Root='Form') if ion_form.Mass < 200
Composition(Name='Form-Am.K-H.H+', Count=1, Mass=67.942081, Root='Form') if ion_form.Mass < 200
Composition(Name='Form-H2O.K41-H.H+', Count=1, Mass=68.95618400000001, Root='Form') if ion_form.Mass < 200
Composition(Name='Form.Na-H.H+', Count=1, Mass=68.99469300000001, Root='Form') if ion_form.Mass < 200
Composition(Name='Form-Am.K41-H.H+', Count=1, Mass=69.9402, Root='Form') if ion_form.Mass < 200
Composition(Name='Form-H